In [20]:
import importlib_resources
import numpy as np
import xlrd


# Load xls sheet with data
doc = xlrd.open_workbook("2019.xls").sheet_by_index(0)





In [5]:
import csv

# Specify the file path
file_path = 'Data/2015.csv'
doc = []
# Open the CSV file
with open(file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Each row is a list representing the columns in that row
        doc.append(row)

# Extract attribute names (1st row, column 4 to 12)
attributeNames = doc.row_values(0, 3, 11)


[['Country',
  'Region',
  'Happiness Rank',
  'Happiness Score',
  'Standard Error',
  'Economy (GDP per Capita)',
  'Family',
  'Health (Life Expectancy)',
  'Freedom',
  'Trust (Government Corruption)',
  'Generosity',
  'Dystopia Residual'],
 ['Switzerland',
  'Western Europe',
  '1',
  '7.587',
  '0.03411',
  '1.39651',
  '1.34951',
  '0.94143',
  '0.66557',
  '0.41978',
  '0.29678',
  '2.51738'],
 ['Iceland',
  'Western Europe',
  '2',
  '7.561',
  '0.04884',
  '1.30232',
  '1.40223',
  '0.94784',
  '0.62877',
  '0.14145',
  '0.4363',
  '2.70201'],
 ['Denmark',
  'Western Europe',
  '3',
  '7.527',
  '0.03328',
  '1.32548',
  '1.36058',
  '0.87464',
  '0.64938',
  '0.48357',
  '0.34139',
  '2.49204'],
 ['Norway',
  'Western Europe',
  '4',
  '7.522',
  '0.0388',
  '1.459',
  '1.33095',
  '0.88521',
  '0.66973',
  '0.36503',
  '0.34699',
  '2.46531'],
 ['Canada',
  'North America',
  '5',
  '7.427',
  '0.03553',
  '1.32629',
  '1.32261',
  '0.90563',
  '0.63297',
  '0.32957',
  '0